In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

In [2]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

In [3]:
#Cargo los datasets
df_1 = pd.read_csv('tp-2020-2c-train-cols1.csv')
df_2 = pd.read_csv('tp-2020-2c-train-cols2.csv')

Recibimos de la compañía FiuMark dos datasets, los cuales fueron importados a los dataframes ```df_1``` y ```df_2```

Notamos que ambos contienen 801 filas, y que tienen en común el campo ```id_usuario```

In [4]:
df_1.shape

(801, 3)

In [5]:
df_1.head()

,Unnamed: 0,id_usuario,volveria
0,116,117,0
1,657,658,0
2,793,794,0
3,454,455,0
4,172,173,1


In [6]:
df_2.shape

(801, 12)

In [7]:
df_2.head()

,Unnamed: 0,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede
0,116,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes
1,657,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes
2,793,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita
3,454,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo
4,172,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo


Intentamos mediante el campo ```id_usuario``` obtener un dataset consolidado a partir de los dos originales. Notamos que la cantidad de filas es la misma.

In [8]:
df_full = df_2.merge(df_1, on='id_usuario')
df_full.shape

(801, 14)

In [9]:
df_full.head()

,Unnamed: 0_x,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede,Unnamed: 0_y,volveria
0,116,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes,116,0
1,657,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes,657,0
2,793,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita,793,0
3,454,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo,454,0
4,172,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo,172,1


Notando que las columnas Unnamed: 0_x y Unnamed: 0_y contienen el id de usuario menos uno, consideramos descartarlas.
Generamos ```df_full_1``` sin dichas columnas.

In [10]:
df_full_1 = df_full.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
df_full_1.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede,volveria
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes,0
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes,0
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita,0
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo,0
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo,1


Con el dataset unificado, inspeccionamos los datos en busqueda de valores nulos y/o sin sentido.

**Preprocesamiento**

Hallamos que las columnas ```edad```, ```fila``` y ```nombre_sede``` contienen valores explicitamente nulos.

In [11]:
for col in df_full_1.columns:
    if df_full_1[col].hasnans:
        print(col + " tiene valores nulos")

edad tiene valores nulos
fila tiene valores nulos
nombre_sede tiene valores nulos


Determinamos qué proporción de valores en dichos campos son nulos:

In [12]:
df_full_1['edad'].isnull().value_counts(normalize=True)*100

False    80.024969
True     19.975031
Name: edad, dtype: float64

In [13]:
df_full_1['fila'].isnull().value_counts(normalize=True)*100

True     77.902622
False    22.097378
Name: fila, dtype: float64

In [14]:
df_full_1['nombre_sede'].isnull().value_counts(normalize=True)*100

False    99.750312
True      0.249688
Name: nombre_sede, dtype: float64

En este caso vemos además la cantidad exacta:

In [15]:
df_full_1['nombre_sede'].isnull().value_counts()

False    799
True       2
Name: nombre_sede, dtype: int64